In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import make_pipeline

# Data Processing

In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, images_test = list(), list()
    spots, spots_test = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = np.array(train_images[slide_name])
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
        # Test 이미지를 불러옵니다.
        test_images = h5file["images/Test"]
        test_spots = h5file["spots/Test"]
        sample = 'S_7'
        image = np.array(test_images[sample])
        p1 = 2000 - image.shape[0]
        p2 = 2000 - image.shape[1]
        images_test.append(np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge'))
        spots_test.append(pd.DataFrame(np.array(test_spots[sample])).assign(slide = 0))
    # EfficientNet의 형식으로 바꿉니다.
    with tf.device('/CPU:0'):
        images = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images))
    df_spots = pd.concat(spots)
    with tf.device('/CPU:0'):
        images_test = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images_test))
    df_spots_test = pd.concat(spots_test)
    return images, df_spots, images_test, df_spots_test

def make_img_proc_info(df, img_with, img_height):
    """
    
    """
    return df.assign(
        left = lambda x: x['x'] - img_width // 2,
        right = lambda x: x['x'] + img_width // 2,
        top = lambda x: x['y'] - img_height // 2,
        bottom = lambda x: x['y'] + img_height // 2,
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )

def create_tf_ds(df):
    if (pd.Series(targets).isin(df.columns)).all():
        return tf.data.Dataset.from_tensor_slices(
            ({
                i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
            }, df[targets])
        )
    else:
        return tf.data.Dataset.from_tensor_slices({
            i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
        })

def proc_images(X):
    return tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1
    )

augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

images, df_spots, images_test, df_spots_test = load_data("data/elucidata_ai_challenge_data.h5")
targets = [i for i in df_spots.columns if i.startswith('C')]

target_proc = make_pipeline(FunctionTransformer(np.log, np.exp),  MinMaxScaler((-1, 1)))
target_proc.fit(df_spots[targets])
df_spots[targets] = target_proc.transform(df_spots[targets])

In [3]:
img_width = 112
img_height = 112

df_spots = make_img_proc_info(df_spots, img_width, img_height)
df_spots_test = make_img_proc_info(df_spots_test, img_width, img_height)

In [4]:
input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)

In [5]:
df_spots['slide'].unique()

array([0, 1, 2, 3, 4, 5])

# Validation

In [6]:
from tqdm.notebook import tqdm
class TqdmEpochProgress(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super().__init__()
        self.epochs = epochs
        self.progress_bar = None

    def on_train_begin(self, logs=None):
        self.progress_bar = tqdm(total=self.epochs, desc="Epochs")

    def on_epoch_end(self, epoch, logs=None):
        log_str = f"loss: {logs.get('loss'):.4f}"
        if 'val_loss' in logs:
            log_str += f", val_loss: {logs.get('val_loss'):.4f}"
        self.progress_bar.set_postfix_str(log_str)
        self.progress_bar.update(1)

    def on_train_end(self, logs=None):
        self.progress_bar.close()

In [8]:
batch_size = 128
epoch = 20
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error
scores = list()
for i, t in enumerate(targets):
    ds_cv_train = create_tf_ds(
        df_spots.loc[df_spots['slide'] != 5].pipe(
            lambda x: pd.concat([
                x, x.sample(n = batch_size - (len(x) % batch_size))
            ])
        )
    ).shuffle(5000).map(
        lambda X, Y: (proc_images(X), Y[i:(i + 1)])
    ).map(
        lambda X, Y: (data_augmentation(X), Y)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

    ds_valid = create_tf_ds(df_spots.loc[df_spots['slide'] == 5]).map(
        lambda X, Y: (proc_images(X), Y[i:(i + 1)])
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

    inputs = tf.keras.Input(shape = input_shape)
    x = enet(inputs, training = False)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')(x)
    outputs = tf.keras.layers.Dense(len(targets))(x)
    m = tf.keras.models.Model(inputs, outputs)

    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        alpha=0.1
    )

    m.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
        metrics = [tf.keras.metrics.MeanSquaredError()]
    )
    hist = m.fit(
        ds_cv_train, validation_data = ds_valid, epochs = epoch,
        callbacks=[TqdmEpochProgress(epoch)], verbose=0 
    )

    s_prd = m.predict(
        create_tf_ds(df_spots.loc[df_spots['slide'] != 5]).map(
            lambda X, Y: proc_images(X)
        ).batch(batch_size), verbose = 0
    )
    scores.append(
        (
            spearmanr(
                df_spots.loc[df_spots['slide'] != 5, t], s_prd[:, 0]
            )[0], 
            mean_squared_error(
                df_spots.loc[df_spots['slide'] != 5, t], s_prd[:, 0]
            )
        )
    )
    print(t, scores[-1])

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

C1 (0.8136341514511527, 0.034391577968181555)


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

C2 (0.8107484381663665, 0.025165876105315357)


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

...

C33 (0.7788554447247308, 0.04571201078348289)


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

C34 (0.8242787907991082, 0.03151513528054646)


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

C35 (0.8952387713244908, 0.01646252496457182)


63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step


(SignificanceResult(statistic=0.887301869957662, pvalue=0.0),
 0.038764171670458564)

In [32]:
np.min(s_prd[:, 0]), np.max(s_prd[:, 0]), np.min(s_prd[:, 0])

(0.45728374, 2.5303652, 0.45728374)

In [28]:
df_spots.loc[df_spots['slide'] != 5, 'C2'].agg(['max', 'min'])

max    1.0
min   -1.0
Name: C2, dtype: float64

# Fine Tuning

과적합을 유의해야하는 데이터셋으로 판단되고 큰 도움은 되지 않습니다.

In [20]:
"""enet.trainable = True
m.compile(
    optimizer=tf.keras.optimizers.Adam(3e-6),  # Low learning rate
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
hist = m.fit(ds_cv_train, validation_data = ds_valid, epochs = 3)
"""

# Train

In [12]:
batch_size = 32
ds_train = create_tf_ds(
    df_spots.pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')(x)
outputs = tf.keras.layers.Dense(len(targets))(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-6,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [tf.keras.metrics.MeanSquaredError()]
)
hist = m.fit(ds_train, epochs = 30)

Epoch 1/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 77s 179ms/step - loss: 0.1945 - mean_squared_error: 0.1945
Epoch 2/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.1452 - mean_squared_error: 0.1452
Epoch 3/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.1300 - mean_squared_error: 0.1300
Epoch 4/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - loss: 0.1221 - mean_squared_error: 0.1221
Epoch 5/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - loss: 0.1170 - mean_squared_error: 0.1170
Epoch 6/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 0.1131 - mean_squared_error: 0.1131
Epoch 7/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.1099 - mean_squared_error: 0.1099
Epoch 8/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - loss: 0.1072 - mean_squared_error: 0.1072
Epoch 9/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.1050 - mean_squared_error: 0.1050
Epoch 10/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.1033 - mean_squared_error: 0.1033
Epoch 11/30
261/26

In [18]:
joblib.dump(m, 'model/eff_b0.joblib')

['model/eff_b0.pkl']

In [4]:
joblib.dump(target_proc, 'model/target_proc.joblib') 

['model/target_proc.joblib']

In [21]:
ds_test = create_tf_ds(df_spots_test)

df_submission = pd.DataFrame(
    np.exp(
        m.predict(
            ds_test.map(lambda X: proc_images(X)).batch(32)
        )
    ), columns = targets
).reset_index().rename(columns = {'index': 'ID'})

33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step


In [22]:
df_submission

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,0.106880,0.117950,0.103375,0.005968,0.003798,0.024129,0.035309,0.008406,0.098681,...,0.013731,0.019819,0.012259,0.018550,0.002482,0.061982,0.069548,0.009430,0.010013,0.059851
1,1,0.128045,0.116623,0.138590,0.010258,0.007435,0.050321,0.042165,0.009597,0.204422,...,0.015731,0.056274,0.016234,0.026071,0.001425,0.072266,0.080541,0.011042,0.010376,0.076468
2,2,0.099630,0.098611,0.100010,0.008563,0.018887,0.026819,0.027433,0.006014,0.137513,...,0.006670,0.036991,0.006439,0.011987,0.001200,0.062143,0.067901,0.003298,0.007506,0.039391
3,3,0.180319,0.137755,0.158754,0.006162,0.006356,0.041420,0.081333,0.030389,0.120627,...,0.032568,0.036748,0.034174,0.035257,0.004906,0.114594,0.134391,0.015988,0.024055,0.124492
4,4,0.087845,0.145078,0.084427,0.001418,0.001762,0.010620,0.054116,0.025442,0.053148,...,0.036336,0.012387,0.023168,0.025999,0.013443,0.077650,0.096791,0.019800,0.012882,0.080666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,0.076367,0.124544,0.080420,0.005161,0.014594,0.023217,0.056135,0.016682,0.076528,...,0.019992,0.027192,0.016148,0.040179,0.002422,0.089227,0.089570,0.010097,0.009481,0.072841
2084,2084,0.109420,0.134732,0.117060,0.002985,0.003336,0.030402,0.056009,0.022790,0.135976,...,0.039139,0.036089,0.032287,0.035312,0.006070,0.085672,0.118656,0.019103,0.013551,0.114055
2085,2085,0.149589,0.089491,0.120247,0.006303,0.009186,0.106010,0.041300,0.011640,0.279060,...,0.021469,0.122370,0.035616,0.024045,0.000779,0.060093,0.092103,0.008761,0.014749,0.097737
2086,2086,0.088822,0.163909,0.087320,0.001273,0.002170,0.009971,0.064393,0.025293,0.044465,...,0.043730,0.010352,0.019291,0.025303,0.020673,0.084443,0.089774,0.022668,0.014188,0.087110
